## This notebook is to do missing data imputation for Train Set

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows',2000)
pd.set_option('display.max_columns',50)
pd.set_option('display.width',1000)

from pprint import pprint
from feature_engine import missing_data_imputers as mdi

import warnings
warnings.filterwarnings("ignore")

### Load Data and Split

In [2]:
df = pd.read_csv('train_numeric_original.csv')

In [3]:
df.head()

,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
0,3876,408,24.583,NaN,18.380,0.945945,0.010761,0.026038,0.005685,0.005635,0.0,0.005935,0.000000,4.172,0.550,0.023,0.107,0.062,0.310,0.166,0.100,NaN,0.262,0.342,11.022908,NaN,NaN,217,NaN,NaN,0.471,0.218,0.190,0.083250,0.327984,0.389168,0.199599,10.008982,9.752342,577
1,10224,1166,28.346,3.0,26.694,0.808959,0.002284,0.173426,0.001803,0.001493,0.0,0.012033,0.000000,7.714,0.545,0.040,0.324,0.201,0.253,0.102,0.066,0.070,0.200,0.148,9.904099,NaN,18.26,181,2009.0,1690.0,0.472,0.290,0.077,0.099099,0.202202,0.302302,0.396396,14.482893,3.013219,844
2,27023,2927,21.641,9.0,31.028,0.956621,0.005141,0.013088,0.004728,0.004708,0.0,0.014809,0.000905,9.789,0.508,0.063,0.129,0.032,0.296,0.215,0.087,0.070,0.187,0.258,11.011502,NaN,19.94,234,4650.0,1350.0,0.496,0.208,0.192,0.137725,0.387226,0.255489,0.219561,9.620346,10.873572,700
3,8735,1039,23.110,0.0,27.734,0.894835,0.000298,0.076576,0.000995,0.004297,0.0,0.022999,0.000000,13.735,0.545,0.038,0.207,0.084,0.255,0.182,0.104,0.081,0.169,0.274,12.845770,NaN,30.31,231,4420.0,1749.0,0.503,0.193,0.215,0.077154,0.308617,0.398798,0.215431,9.383225,10.870809,592
4,3681,365,21.985,2.0,19.673,0.923886,0.000496,0.042889,0.000000,0.013622,0.0,0.016905,0.002202,8.217,0.474,0.034,0.177,0.075,0.324,0.118,0.113,NaN,0.178,0.340,11.565750,NaN,NaN,270,NaN,1249.0,0.516,0.222,0.230,0.092537,0.362189,0.346269,0.199005,9.543286,14.723926,444


In [4]:
df.tail()

,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
1557,18983,1480,22.621,41.0,26.905,0.847652,0.064786,0.048670,0.000000,0.012277,0.001800,0.024815,0.000000,5.699,0.517,0.044,0.216,0.086,0.302,NaN,0.096,0.079,NaN,0.227,12.942281,NaN,28.94,207,6299.0,4660.0,0.510,0.243,0.144,0.153614,0.347390,0.263052,0.235944,11.369618,7.211285,928
1558,18837,2144,28.649,7.0,26.515,0.925500,0.010666,0.028790,0.000398,0.020893,0.001085,0.012667,0.000000,6.789,0.559,0.043,0.112,0.038,0.294,0.169,0.081,0.060,0.166,0.242,10.669456,NaN,18.02,272,2030.0,1249.0,0.510,0.205,0.193,0.070858,0.371257,0.311377,0.246507,9.330009,12.600874,640
1559,77224,4677,18.928,137.0,28.827,0.883936,0.070858,0.019081,0.000614,0.012569,0.000302,0.011835,0.000805,6.294,0.498,0.051,0.172,0.071,0.260,0.193,0.096,0.080,0.168,0.298,12.962100,NaN,13.78,237,6979.0,1529.0,0.503,0.211,0.179,0.118644,0.311067,0.302094,0.268195,9.029286,8.846613,754
1560,4698,402,20.107,1.0,48.980,0.173909,0.000911,0.824579,0.000401,0.000000,0.000000,0.000000,0.000200,17.300,0.493,0.112,0.273,0.129,0.235,0.171,0.099,0.110,0.131,0.246,10.244994,NaN,NaN,192,4710.0,NaN,0.484,0.195,0.206,0.164000,0.354000,0.342000,0.140000,8.445214,9.744478,640
1561,650813,92680,37.789,NaN,30.687,0.704111,0.057262,0.161265,0.003829,0.027315,0.003196,0.041223,0.001799,8.343,0.462,0.060,0.185,0.084,0.216,0.178,0.059,0.100,0.149,0.174,13.099012,4.2,9.88,218,1069.0,1749.0,0.504,0.255,0.108,0.061876,0.208583,0.370259,0.359281,13.944628,6.173158,976


In [5]:
df.dtypes

population                                      int64
renter_occupied_households                      int64
pct_renter_occupied                           float64
evictions                                     float64
rent_burden                                   float64
pct_white                                     float64
pct_af_am                                     float64
pct_hispanic                                  float64
pct_am_ind                                    float64
pct_asian                                     float64
pct_nh_pi                                     float64
pct_multiple                                  float64
pct_other                                     float64
poverty_rate                                  float64
pct_civilian_labor                            float64
pct_unemployment                              float64
pct_uninsured_adults                          float64
pct_uninsured_children                        float64
pct_adult_obesity           

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562 entries, 0 to 1561
Data columns (total 40 columns):
population                                    1562 non-null int64
renter_occupied_households                    1562 non-null int64
pct_renter_occupied                           1562 non-null float64
evictions                                     1235 non-null float64
rent_burden                                   1562 non-null float64
pct_white                                     1562 non-null float64
pct_af_am                                     1562 non-null float64
pct_hispanic                                  1562 non-null float64
pct_am_ind                                    1562 non-null float64
pct_asian                                     1562 non-null float64
pct_nh_pi                                     1562 non-null float64
pct_multiple                                  1562 non-null float64
pct_other                                     1562 non-null float64
poverty_rate 

In [7]:
df.describe()

,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
count,1.562000e+03,1.562000e+03,1562.000000,1235.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1560.000000,1560.000000,1560.000000,1344.000000,1560.000000,1446.000000,1100.000000,1560.000000,1542.000000,613.000000,1372.000000,1562.000000,1447.000000,1448.000000,1560.000000,1560.000000,1560.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000
mean,1.083407e+05,1.490462e+04,28.525570,397.410526,28.537526,0.769045,0.088593,0.092012,0.018431,0.012709,0.000711,0.017583,0.000916,12.182780,0.470535,0.062551,0.220037,0.088844,0.304546,0.211682,0.106566,0.083326,0.164818,0.276981,11.637336,5.752414,21.715153,275.482714,3421.828611,2508.303867,0.499388,0.228672,0.167707,0.145666,0.346271,0.303030,0.205033,11.621356,10.415138,701.142125
std,3.745229e+05,6.255947e+04,8.121601,1522.801074,4.670467,0.203306,0.143505,0.141599,0.074762,0.026693,0.003059,0.015836,0.001657,5.783889,0.070936,0.022615,0.067500,0.041480,0.043550,0.064045,0.022521,0.021239,0.051483,0.053098,1.534144,4.297808,10.721369,57.827540,2538.670834,1960.312344,0.023933,0.034732,0.044555,0.067483,0.071152,0.052236,0.092001,2.756009,2.772070,192.883110
min,2.690000e+02,6.400000e+01,7.279000,-1.000000,9.909000,0.009547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.186000,0.012000,0.053000,0.018000,0.133000,0.031000,0.033000,0.030000,0.032000,0.104000,7.209413,-0.080000,3.140000,76.000000,340.000000,279.000000,0.314000,0.082000,0.036000,0.019000,0.074297,0.114458,0.064128,3.654080,0.961076,351.000000
25%,1.045275e+04,1.078250e+03,22.874000,3.000000,25.869000,0.649967,0.005523,0.018747,0.001201,0.002391,0.000000,0.009465,0.000000,7.991000,0.425000,0.046000,0.171750,0.059000,0.283000,0.169000,0.092000,0.070000,0.129000,0.243000,10.432910,2.710000,14.047500,233.000000,1859.000000,1389.750000,0.495000,0.208000,0.140000,0.094226,0.300601,0.269461,0.142142,10.014818,8.613691,578.000000
50%,2.528200e+04,2.754000e+03,27.199500,27.000000,28.768000,0.846542,0.020599,0.037613,0.002610,0.005279,0.000000,0.014438,0.000302,11.173500,0.470500,0.061000,0.216000,0.079000,0.306000,0.206000,0.105000,0.080000,0.163000,0.281000,11.907300,4.540000,20.285000,270.000000,2730.000000,1969.500000,0.504000,0.227000,0.164000,0.129388,0.352176,0.303303,0.182365,11.435441,10.396898,650.000000
75%,6.836150e+04,7.987000e+03,32.341500,162.500000,31.307250,0.933479,0.102047,0.092998,0.006121,0.011886,0.000403,0.020999,0.001197,15.018500,0.515000,0.076000,0.265000,0.109000,0.331250,0.249000,0.122000,0.091000,0.196000,0.312000,12.885680,7.600000,27.072500,311.000000,4064.500000,2840.000000,0.511000,0.246000,0.191000,0.187719,0.396556,0.337672,0.241234,12.940235,12.250655,773.750000
max,1.002029e+07,1.760277e+06,73.008000,29251.000000,49.665000,0.995141,0.756011,0.987039,0.815817,0.418088,0.085037,0.184332,0.019582,38.792000,0.996000,0.242000,0.520000,0.327000,0.474000,0.513000,0.180000,0.182000,0.419000,0.446000,14.992477,26.920000,110.450000,511.000000,25169.000000,16740.000000,0.564000,0.415000,0.488000,0.535750,0.535536,0.477341,0.788153,29.034900,24.281150

In [8]:
df.shape

(1562, 40)

### Treat Missing Values using Feature-Engine

In [9]:
df.isnull().sum()

population                                      0
renter_occupied_households                      0
pct_renter_occupied                             0
evictions                                     327
rent_burden                                     0
pct_white                                       0
pct_af_am                                       0
pct_hispanic                                    0
pct_am_ind                                      0
pct_asian                                       0
pct_nh_pi                                       0
pct_multiple                                    0
pct_other                                       0
poverty_rate                                    0
pct_civilian_labor                              0
pct_unemployment                                0
pct_uninsured_adults                            2
pct_uninsured_children                          2
pct_adult_obesity                               2
pct_adult_smoking                             218


In [10]:
imputer = mdi.MeanMedianImputer(imputation_method='mean',variables=None)

In [11]:
imputer.fit(df)

MeanMedianImputer(imputation_method='mean',
                  variables=['population', 'renter_occupied_households',
                             'pct_renter_occupied', 'evictions', 'rent_burden',
                             'pct_white', 'pct_af_am', 'pct_hispanic',
                             'pct_am_ind', 'pct_asian', 'pct_nh_pi',
                             'pct_multiple', 'pct_other', 'poverty_rate',
                             'pct_civilian_labor', 'pct_unemployment',
                             'pct_uninsured_adults', 'pct_uninsured_children',
                             'pct_adult_obesity', 'pct_adult_smoking',
                             'pct_diabetes', 'pct_low_birthweight',
                             'pct_excessive_drinking',
                             'pct_physical_inactivity',
                             'air_pollution_particulate_matter_value',
                             'homicides_per_100k',
                             'motor_vehicle_crash_deaths_per_100k',

In [12]:
df1 = imputer.transform(df)

In [13]:
df1.isnull().sum()

population                                    0
renter_occupied_households                    0
pct_renter_occupied                           0
evictions                                     0
rent_burden                                   0
pct_white                                     0
pct_af_am                                     0
pct_hispanic                                  0
pct_am_ind                                    0
pct_asian                                     0
pct_nh_pi                                     0
pct_multiple                                  0
pct_other                                     0
poverty_rate                                  0
pct_civilian_labor                            0
pct_unemployment                              0
pct_uninsured_adults                          0
pct_uninsured_children                        0
pct_adult_obesity                             0
pct_adult_smoking                             0
pct_diabetes                            

In [14]:
df1

,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
0,3876,408,24.583,397.410526,18.380,0.945945,0.010761,0.026038,0.005685,0.005635,0.000000,0.005935,0.000000,4.172,0.550,0.023,0.107000,0.062000,0.310000,0.166000,0.100000,0.083326,0.262000,0.342000,11.022908,5.752414,21.715153,217,3421.828611,2508.303867,0.471000,0.218000,0.190000,0.083250,0.327984,0.389168,0.199599,10.008982,9.752342,577
1,10224,1166,28.346,3.000000,26.694,0.808959,0.002284,0.173426,0.001803,0.001493,0.000000,0.012033,0.000000,7.714,0.545,0.040,0.324000,0.201000,0.253000,0.102000,0.066000,0.070000,0.200000,0.148000,9.904099,5.752414,18.260000,181,2009.000000,1690.000000,0.472000,0.290000,0.077000,0.099099,0.202202,0.302302,0.396396,14.482893,3.013219,844
2,27023,2927,21.641,9.000000,31.028,0.956621,0.005141,0.013088,0.004728,0.004708,0.000000,0.014809,0.000905,9.789,0.508,0.063,0.129000,0.032000,0.296000,0.215000,0.087000,0.070000,0.187000,0.258000,11.011502,5.752414,19.940000,234,4650.000000,1350.000000,0.496000,0.208000,0.192000,0.137725,0.387226,0.255489,0.219561,9.620346,10.873572,700
3,8735,1039,23.110,0.000000,27.734,0.894835,0.000298,0.076576,0.000995,0.004297,0.000000,0.022999,0.000000,13.735,0.545,0.038,0.207000,0.084000,0.255000,0.182000,0.104000,0.081000,0.169000,0.274000,12.845770,5.752414,30.310000,231,4420.000000,1749.000000,0.503000,0.193000,0.215000,0.077154,0.308617,0.398798,0.215431,9.383225,10.870809,592
4,3681,365,21.985,2.000000,19.673,0.923886,0.000496,0.042889,0.000000,0.013622,0.000000,0.016905,0.002202,8.217,0.474,0.034,0.177000,0.075000,0.324000,0.118000,0.113000,0.083326,0.178000,0.340000,11.565750,5.752414,21.715153,270,3421.828611,1249.000000,0.516000,0.222000,0.230000,0.092537,0.362189,0.346269,0.199005,9.543286,14.723926,444
5,8540,751,22.543,0.000000,34.937,0.560637,0.417547,0.000597,0.001294,0.000000,0.000000,0.019925,0.000000,21.051,0.361,0.110,0.246000,0.100000,0.322000,0.211682,0.148000,0.129000,0.164818,0.317000,12.822859,5.752414,39.160000,388,3421.828611,8770.000000,0.515000,0.199000,0.185000,0.317000,0.388000,0.198000,0.097000,10.456274,14.139734,531
6,22712,1901,24.602,19.000000,29.242,0.929799,0.005564,0.039416,0.002389,0.005106,0.000000,0.016234,0.001493,9.651,0.470,0.061,0.258000,0.126000,0.275000,0.294000,0.108000,0.070000,0.299000,0.231000,13.395494,5.752414,26.490000,254,2500.000000,1489.000000,0.494000,0.218000,0.162000,0.142572,0.287139,0.292124,0.278166,10.953481,6.818432,976
7,27974,3873,38.012,49.000000,29.966,0.537084,0.008489,0.396900,0.017048,0.024534,0.002472,0.013473,0.000000,15.655,0.462,0.106,0.297000,0.117000,0.267000,0.150000,0.078000,0.050000,0.169000,0.188000,8.035606,5.752414,20.630000,239,1410.000000,7030.000000,0.497000,0.278000,0.141000,0.249004,0.293825,0.307769,0.149402,14.526719,9.660626,725
8,2027,295,55.572,397.410526,21.808,0.184899,0.001981,0.010408,0.749382,0.000000,0.016572,0.036758,0.000000,36.120,0.353,0.077,0.316000,0.086000,0.421000,0.419000,0.177000,0.070000,0.274000,0.384000,10.904799,5.752414,21.715153,511,670.000000,1980.000000,0.500000,0.415000,0.075000,0.181456,0.432702,0.291127,0.094716,26.816187,6.825939,533
9,17165,2223,29.769,397.410526,28.664,0.961332,0.009077,0.011856,0.002779,0.000800,0.000000,0.014056,0.000100,17.793,0.410,0.073,0

In [15]:
df1.shape

(1562, 40)

### Save a copy

In [16]:
#df1.to_csv('trainfilled.csv',index=False)